In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
import urllib.request
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
from io import StringIO
from matplotlib import pyplot as plt
from datetime import timedelta
from datetime import datetime
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [ ]:
def get_melbourne_data() -> pd.DataFrame:
    '''
    Returns a dataframe of the melbourne data set.
    :return: pd.DataFrame
    '''

    # URL of the raw csv data to download
    raw_url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-min-temperatures.csv"

    # Get the earthquake data from the API
    response = urllib.request.urlopen(raw_url)

    # Decode earthquake data
    response = response.read().decode('utf-8')

    # Return as a pandas dataframe
    data = pd.read_csv(StringIO(response))

    # Cast the date column to datetime
    data['Date'] = pd.to_datetime(data['Date'])

    return data


def split_train_test_data(melbourne_data: pd.DataFrame, split_year: str="1987") -> (pd.DataFrame, pd.DataFrame):
    '''
    Split the melbourne data into a training dataframe and a test dataframe.
    The training data is composed of all temperature points strictly anterior to the given split year.
    The test data is composed of all the points posterior or equal to the split year.
    :param melbourne_data: pd.DataFrame, with at least column ['Date']
    :param split_year: str, the year to split the data on
    :return: (pd.DataFrame, pd.DataFrame)
    '''

    # Format split year variable
    split_year = "{}".format(int(split_year) - 1)

    # Trainings data. Data anterior to the given split year
    train_data = melbourne_data.loc[:split_year]

    # Test data. Data posterior or equal to the given split year
    test_data = melbourne_data.loc[split_year:]

    return train_data, test_data

def build_training_point(data, t_str, history_days=64, horizon_days=1):
    '''
    :param data:
    :param t_str:
    :param history_days:
    :param horizon_days:
    :return:
    '''

    # Cast for indexing
    t_datetime = datetime.strptime(t_str, "%Y-%m-%d 00:00:00")

    # Create training example (x,y)
    try:
        x = data.loc[t_datetime - timedelta(days=history_days - 1):t_datetime]
        y = data.loc[t_datetime + timedelta(days=1):t_datetime + timedelta(days=horizon_days)]
    except KeyError:
        raise KeyError("The date {} is not in the data".format(t_str))

    # Return
    return x, y


def create_training_points(data, history_days=64, horizon_days=32):
    '''
    :param data:
    :param history_days:
    :param horizon_days:
    :return:
    '''
    X = []
    Y = []
    for t in data.index[history_days:(len(data) - horizon_days)]:
        try:
            x, y = build_training_point(data, str(t), history_days=history_days, horizon_days=horizon_days)
            if (len(x) == history_days) & (len(y) == horizon_days):
                X.append(x)
                Y.append(y)
        except KeyError:
            continue
    X = np.stack(X)
    Y = np.stack(Y)
    return X, Y

melbourne_data = get_melbourne_data()

In [ ]:
print(melbourne_data.head())

        Date  Temp
0 1981-01-01  20.7
1 1981-01-02  17.9
2 1981-01-03  18.8
3 1981-01-04  14.6
4 1981-01-05  15.8


In [ ]:
def split_data(year):
  split_date = pd.datetime(year,1,1)
  train_data = melbourne_data.loc[melbourne_data['Date'] < split_date]
  test_data = melbourne_data.loc[melbourne_data['Date'] >= split_date]

  scaler = MinMaxScaler()
  train_data[["Temp"]] = scaler.fit_transform(train_data[["Temp"]] )
  test_data[["Temp"]] = scaler.fit_transform(test_data[["Temp"]] )

  # Index
  train_data.set_index('Date', inplace=True)
  test_data.set_index('Date', inplace=True)
  return train_data, test_data

In [ ]:
from keras.layers import Input, LSTM, Dense, Flatten
from keras.models import Model

def create_model(history_days, horizon_days):
  model_input = Input(shape=(history_days, 1), name='x', dtype='float32')
  z = LSTM(64, activation='relu', return_sequences=True)(model_input)
  z = Flatten()(z) 
  z = Dense(horizon_days, activation='linear')(z)

  # Keras model
  model_keras = Model(inputs=model_input, outputs=z)

  return model_keras

model_3m = create_model(90, 30)
model_6m = create_model(180, 30)
model_12m = create_model(365, 30)

In [ ]:
train_data, test_data = split_data(1987)
X_train, Y_train = create_training_points(train_data, history_days=90, horizon_days=30)
model_3m.compile(optimizer='adam', loss='mse')
model_3m.fit(X_train, Y_train, epochs=32, batch_size=32, verbose=1)

Epoch 1/32
61/61 [==============================] - 4s 47ms/step - loss: 0.0283
Epoch 2/32
61/61 [==============================] - 3s 48ms/step - loss: 0.0131
Epoch 3/32
61/61 [==============================] - 3s 54ms/step - loss: 0.0125
Epoch 4/32
61/61 [==============================] - 3s 49ms/step - loss: 0.0124
Epoch 5/32
61/61 [==============================] - 3s 48ms/step - loss: 0.0123
Epoch 6/32
61/61 [==============================] - 4s 66ms/step - loss: 0.0122
Epoch 7/32
61/61 [==============================] - 3s 57ms/step - loss: 0.0121
Epoch 8/32
61/61 [==============================] - 3s 48ms/step - loss: 0.0121
Epoch 9/32
61/61 [==============================] - 3s 47ms/step - loss: 0.0120
Epoch 10/32
61/61 [==============================] - 3s 47ms/step - loss: 0.0120
Epoch 11/32
61/61 [==============================] - 3s 47ms/step - loss: 0.0121
Epoch 12/32
61/61 [==============================] - 3s 47ms/step - loss: 0.0118
Epoch 13/32
61/61 [==================

In [ ]:
train_data, test_data = split_data(1987)
X_test, Y_test = create_training_points(test_data, history_days=90, horizon_days=30)
y_pred_3m = model_3m.predict(X_test)
print("1987, k=3: ", mean_squared_error(y_pred_3m[0], Y_test[0]))

train_data, test_data = split_data(1988)
X_test, Y_test = create_training_points(test_data, history_days=90, horizon_days=30)
y_pred_3m = model_3m.predict(X_test)
print("1988, k=3: ", mean_squared_error(y_pred_3m[0], Y_train[1]))

train_data, test_data = split_data(1989)
X_test, Y_test = create_training_points(test_data, history_days=90, horizon_days=30)
y_pred_3m = model_3m.predict(X_test)
print("1989, k=3: ", mean_squared_error(y_pred_3m[0], Y_train[2]))

1987, k=3:  0.027352685494250226
1988, k=3:  0.011883026331488007
1989, k=3:  0.019069123879120797


In [ ]:
train_data, test_data = split_data(1987)
X_train, Y_train = create_training_points(train_data, history_days=180, horizon_days=30)
model_6m.compile(optimizer='adam', loss='mse')
model_6m.fit(X_train, Y_train, epochs=32, batch_size=32, verbose=1)

Epoch 1/32
56/56 [==============================] - 6s 93ms/step - loss: 0.0283
Epoch 2/32
56/56 [==============================] - 5s 92ms/step - loss: 0.0122
Epoch 3/32
56/56 [==============================] - 5s 92ms/step - loss: 0.0120
Epoch 4/32
56/56 [==============================] - 5s 91ms/step - loss: 0.0115
Epoch 5/32
56/56 [==============================] - 5s 91ms/step - loss: 0.0115
Epoch 6/32
56/56 [==============================] - 5s 93ms/step - loss: 0.0111
Epoch 7/32
56/56 [==============================] - 5s 93ms/step - loss: 0.0112
Epoch 8/32
56/56 [==============================] - 5s 93ms/step - loss: 0.0111
Epoch 9/32
56/56 [==============================] - 5s 93ms/step - loss: 0.0108
Epoch 10/32
56/56 [==============================] - 5s 93ms/step - loss: 0.0109
Epoch 11/32
56/56 [==============================] - 5s 93ms/step - loss: 0.0107
Epoch 12/32
56/56 [==============================] - 5s 92ms/step - loss: 0.0107
Epoch 13/32
56/56 [==================

In [ ]:
train_data, test_data = split_data(1987)
X_test, Y_test = create_training_points(test_data, history_days=180, horizon_days=30)
y_pred_6m = model_6m.predict(X_test)
print("1987, k=6: ", mean_squared_error(y_pred_6m[0], Y_test[0]))

train_data, test_data = split_data(1988)
X_test, Y_test = create_training_points(test_data, history_days=180, horizon_days=30)
y_pred_6m = model_6m.predict(X_test)
print("1988, k=6: ", mean_squared_error(y_pred_6m[0], Y_train[1]))

train_data, test_data = split_data(1989)
X_test, Y_test = create_training_points(test_data, history_days=180, horizon_days=30)
y_pred_6m = model_6m.predict(X_test)
print("1989, k=6: ", mean_squared_error(y_pred_6m[0], Y_train[2]))

1987, k=6:  0.007991645052612689
1988, k=6:  0.007169696315602252
1989, k=6:  0.012237317576332769


In [ ]:
train_data, test_data = split_data(1987)
X_train, Y_train = create_training_points(train_data, history_days=365, horizon_days=30)
model_12m.compile(optimizer='adam', loss='mse')
model_12m.fit(X_train, Y_train, epochs=32, batch_size=32, verbose=1)

Epoch 1/32
44/44 [==============================] - 9s 180ms/step - loss: 0.0258
Epoch 2/32
44/44 [==============================] - 8s 181ms/step - loss: 0.0121
Epoch 3/32
44/44 [==============================] - 8s 179ms/step - loss: 0.0118
Epoch 4/32
44/44 [==============================] - 8s 178ms/step - loss: 0.0115
Epoch 5/32
44/44 [==============================] - 8s 176ms/step - loss: 0.0111
Epoch 6/32
44/44 [==============================] - 8s 176ms/step - loss: 0.0109
Epoch 7/32
44/44 [==============================] - 11s 245ms/step - loss: 0.0104
Epoch 8/32
44/44 [==============================] - 10s 234ms/step - loss: 0.0102
Epoch 9/32
44/44 [==============================] - 9s 199ms/step - loss: 0.0099
Epoch 10/32
44/44 [==============================] - 8s 181ms/step - loss: 0.0096
Epoch 11/32
44/44 [==============================] - 8s 179ms/step - loss: 0.0094
Epoch 12/32
44/44 [==============================] - 8s 177ms/step - loss: 0.0092
Epoch 13/32
44/44 [====

In [ ]:
train_data, test_data = split_data(1987)
X_test, Y_test = create_training_points(test_data, history_days=365, horizon_days=30)
y_pred_12m = model_12m.predict(X_test)
print("1987, k=12: ", mean_squared_error(y_pred_12m[0], Y_test[0]))

train_data, test_data = split_data(1988)
X_test, Y_test = create_training_points(test_data, history_days=365, horizon_days=30)
y_pred_12m = model_12m.predict(X_test)
print("1988, k=12: ", mean_squared_error(y_pred_12m[0], Y_train[1]))

train_data, test_data = split_data(1989)
X_test, Y_test = create_training_points(test_data, history_days=365, horizon_days=30)
y_pred_12m = model_12m.predict(X_test)
print("1989, k=12: ", mean_squared_error(y_pred_12m[0], Y_train[2]))

1987, k=12:  0.03865884223545268
1988, k=12:  0.01985055503939519
1989, k=12:  0.04430945823737449


Metrics: Mean Squared Error

1987, k=3:  0.027352685494250226
1988, k=3:  0.011883026331488007
1989, k=3:  0.019069123879120797

1987, k=6:  0.007991645052612689
1988, k=6:  0.007169696315602252
1989, k=6:  0.012237317576332769

1987, k=12:  0.03865884223545268
1988, k=12:  0.01985055503939519
1989, k=12:  0.04430945823737449

<table>
    <thead>
        <tr>
            <th>Evaluation year</th>
            <th>Next 3 months</th>
            <th>Next 6 months</th>
            <th>Next 12 months</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>1987</td>
            <td>0.0273</td>
            <td>0.0079</td>
            <td>0.0386</td>
        </tr>
        <tr>
            <td>1988</td>
            <td>0.0118</td>
            <td>0.0071</td>
            <td>0.0198</td>
        </tr>
        <tr>
            <td>1989</td>
            <td>0.0190</td>
            <td>0.0122</td>
            <td>0.0443</td>
        </tr>
    </tbody>
</table>

Le model le plus précis est le model est k=6. Il serait cependant interessant de voir si une modification de l architecture du réseau puisse améliorer les modèles k=3 et k=12. Le k=12 etant peut etre un peu léger pour retenir une si grande fenetre.